<img src="../assets/header_notebook.png" />
<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>ESA - Black Sea Deoxygenation Emulator</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# ----------
# Librairies
# ----------
import os
import sys
import cv2
import dawgz
import wandb
import xarray
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# Dawgz (jobs //)
from dawgz import job, schedule

# -------------------
# Librairies (Custom)
# -------------------
# Adding path to source folder to load custom modules
sys.path.insert(1, '../src/debs/')
sys.path.insert(1, '../scripts/')

# Loading libraries
from metrics              import *
from tools                import *
from dataset              import *
from dataloader           import *

# -------
# Jupyter
# -------
%matplotlib inline
plt.rcParams.update({'font.size': 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

# Moving to the .py directory
%cd ../src/debs/

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Scripts</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# Training a neural network:
%run training.py    --start_year                 0 \
                    --end_year                   0 \
                    --start_month                0 \
                    --end_month                  1 \
                    --inputs           temperature \
                    --problem       classification \
                    --windows_input              1 \
                    --windows_output             1 \
                    --depth                    200 \
                    --architecture         AVERAGE \
                    --scaling                    1 \
                    --loss_weights             1 1 \
                    --learning_rate          0.001 \
                    --batch_size                64 \
                    --epochs                     5 \
                    --kernel_size                3 \
                    --dawgz                  False

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [14]:
# -----------------
#    Parameters
# -----------------
#
# Time window
month_starting = 1
month_ending   = 2
year_starting  = 1980
year_ending    = 1980

# Maximum depth observed for oxygen, what is left is masked [m] (Note: To observe only the continental shelf set it to ~200m).
depth_max_oxygen = 200

# ------------------
#  Loading the data
# ------------------
# Loading the different datasets
BSD_dataset = BlackSea_Dataset(year_start  = year_starting,
                               year_end    = year_ending,
                               month_start = month_starting,
                               month_end   = month_ending)

# Loading the days ID (used to give temporal information to the model)
days_ID = BSD_dataset.get_days()

# Loading the different field values
data_temperature   = BSD_dataset.get_data(variable = "temperature", region = "surface")
data_salinity      = BSD_dataset.get_data(variable = "salinity",    region = "surface")
data_chlorophyll   = BSD_dataset.get_data(variable = "chlorophyll", region = "surface")
data_kshort        = BSD_dataset.get_data(variable = "kshort",      region = "surface")
data_klong         = BSD_dataset.get_data(variable = "klong",       region = "surface")
data_oxygen        = BSD_dataset.get_data(variable = "oxygen",      region = "bottom", depth = depth_max_oxygen)

# Loading the black sea masks
bs_mask             = BSD_dataset.get_mask(depth = None)
bs_mask_with_depth  = BSD_dataset.get_mask(depth = depth_max_oxygen)
bs_mask_complete    = get_complete_mask(data_oxygen, bs_mask_with_depth)

# --------------------
#  Preparing the data
# --------------------
BSD_loader = BlackSea_Dataloader(x = [data_temperature, data_chlorophyll],
                                 y = data_oxygen,
                                 t = days_ID,
                              mode = "regression",
                              mask = bs_mask,
                   mask_with_depth = bs_mask_with_depth,
                        window_inp = 10,
                        window_out = 1,
                    window_transfo = 1)

# Retrieving the datasets
ds_train      = BSD_loader.get_dataloader("train")
ds_validation = BSD_loader.get_dataloader("validation")
ds_test       = BSD_loader.get_dataloader("test")

In [20]:
# Pytorch
import torch.nn as nn


class Encoder(nn.Sequential):
    r"""A neural network used to encode the temporal information of the data and return weights for the input data"""

    def __init__(self, input_size : int):
        super(Encoder, self).__init__()

        # Defining the layers
        self.linear_in       = nn.Linear(input_size, 256)
        self.linear_middle_1 = nn.Linear(256,        256)
        self.linear_middle_2 = nn.Linear(256,        128)
        self.linear_middle_3 = nn.Linear(128,         64)
        self.linear_middle_4 = nn.Linear(64,          32)
        self.linear_out      = nn.Linear(32,           1)

        # Defining the activation functions
        self.activation = nn.GELU()

        # Defining the softmax function, i.e. (t, values, day) to (t, values, 1) then (t, weights, 1)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, x):

        # Applying the layers
        x = self.activation(self.linear_in(x))
        x = self.activation(self.linear_middle_1(x))
        x = self.activation(self.linear_middle_2(x))
        x = self.activation(self.linear_middle_3(x))
        x = self.activation(self.linear_middle_4(x))
        x = self.linear_out(x)

        # Applying the softmax function
        return self.softmax(x)

    def count_parameters(self,):
        r"""Determines the number of trainable parameters in the model"""
        return int(sum(p.numel() for p in self.parameters() if p.requires_grad))



for x, t, y in ds_train:

    print(x.shape, t.shape, y.shape)

    print(t)
    nn = Encoder(t.shape[-1])

    print(nn.count_parameters())
    res = nn(t)

    print(res.shape)
    print(res[0, :, 0])




    break

torch.Size([24, 10, 1, 2, 258, 578]) torch.Size([24, 10, 1]) torch.Size([24, 1, 258, 578])
tensor([[[ 1.],
         [ 2.],
         [ 3.],
         [ 4.],
         [ 5.],
         [ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.]],

        [[ 2.],
         [ 3.],
         [ 4.],
         [ 5.],
         [ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.],
         [11.]],

        [[ 3.],
         [ 4.],
         [ 5.],
         [ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.],
         [11.],
         [12.]],

        [[ 4.],
         [ 5.],
         [ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.],
         [11.],
         [12.],
         [13.]],

        [[ 5.],
         [ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.],
         [11.],
         [12.],
         [13.],
         [14.]],

        [[ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.],
         [11.],
   